In [75]:
%load_ext autoreload
%autoreload 2

from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
import os
import json

from langsmith import traceable

class MetaDataGrader(BaseModel):
    binary_score: str = Field(description=" ")

# Load the configuration from the config.json file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Set the state variables
os.environ["AWS_ACCESS_KEY_ID"] = config["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["AWS_SECRET_ACCESS_KEY"]

# Usage
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="Agentic-Love"

In [77]:

from langsmith.wrappers import wrap_openai
from langsmith import traceable

import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable

# Auto-trace LLM calls in-context

bedrock_llm = wrap_openai(BedrockLLM(
    model_id="meta.llama3-70b-instruct-v1:0",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key, 
    temperature=0.5,
    max_tokens = 512,
    # model_kwargs= {"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
))

# Create a prompt template
prompt_template = PromptTemplate(
    input_variables=["input"],
    template="Philosophize on this: {input}"
)

# Create the LangChain LLM Chain
transfer_news_chain = bedrock_llm | prompt_template


# # Use the chain
# input_text = "What is chess?"
# result = transfer_news_chain.invoke(input_text)
# result.to_messages()

# Assuming result is a string that corresponds to binary_score
# graded_result = MetaDataGrader(binary_score=result.to_string())
# print(graded_result)

AttributeError: 'BedrockLLM' object has no attribute 'chat'

In [72]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI # Using OpenAI for simplicity, replace with Anthropic if needed.
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
from bs4 import BeautifulSoup

# Wikipedia search tool
@tool
def wikipedia_search(query: str):
    """Searches Wikipedia for a given query and returns a summary."""
    try:
        url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"

        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        content = soup.find('div', class_='mw-parser-output')
        if content:
            paragraphs = content.find_all('p')
            summary = " ".join(paragraph.text for paragraph in paragraphs[:3]).strip()
            return summary
        else:
            return "Error: Content not found on Wikipedia."
    except requests.exceptions.RequestException as e:
        return f"Error: Request failed - {e}"
    except Exception as e:
        return f"Error: An unexpected error occurred - {e}"


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    # Check for tool calls in the *last response*, not the user's message.
    # if len(messages) > 1 and messages[-1].tool_calls:  # Check the AI's response
    #     return "tools"  # Go to tools node if the model suggests a tool call
    if len(messages) > 1 and "Error" not in messages[-1].content and len(messages[-1].content) > 100:
        return END  # Stop if a long enough, error-free summary is obtained
    else:
        return "tools" 

@traceable
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [65]:
tool_node = ToolNode([wikipedia_search])

In [66]:


bedrock_llm = BedrockLLM(
    model_id="meta.llama3-70b-instruct-v1:0",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key, 
    temperature=0.5,
    max_tokens = 512,
    # model_kwargs= {"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
)

#Attempt to bind tools (This might not work directly with BedrockLLM)
try:
    model = bedrock_llm.bind_tools(tools)
except Exception as e:
    print(f"Warning: Binding tools to BedrockLLM failed: {e}.  Tool usage might not function as expected. Attempting to continue without tool binding.")
    model = bedrock_llm #If tool binding fails, proceed without it

# ... (should_continue, call_model, workflow, checkpointer, app remain the same) ...


In [67]:


workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

#Run the agent
final_state = app.invoke(
    {"messages": [HumanMessage(content="Summarize the history of Python")]},
    config={"configurable": {"thread_id": 42}} # Added thread_id
)
print(final_state["messages"][-1].content)



 programming language.

Python was created in the late ed by Guido van Rossum at the National Research Institute for Mathematics and Computer Science in the Netherlands. Here's a brief summary of the history of Python:

* **1991:** Guido van Rossum begins working on Python at the National Research Institute for Mathematics and Computer Science in the Netherlands.
* **1994:** Python 1.0 is released, featuring a simple syntax and a focus on code readability.
* **1995:** Python 1.2 is released, adding support for functional programming and the `lambda` function.
* **1996:** Python 1.3 is released, introducing the `with` statement and improved support for object-oriented programming.
* **1999:** Python 1.5 is released, featuring a new garbage collector and improved performance.
* **2000:** Python 1.6 is released, adding support for Unicode and improved support for Windows.
* **2001:** Python 2.0 is released, featuring a new garbage collector and improved performance.
* **2001-2005:** Pytho